In [23]:
# User visits problem from the rel-avito dataset
# on relbench: https://relbench.stanford.edu/datasets/rel-avito/#user-clicks

# 1) https://demo.kurve.ai
# 2) create graph of rel-avito data
# 3) assign user as parent node with depth 4

In [1]:
import pandas as pd
from torch_frame.utils import infer_df_stype
import catboost
from sklearn import metrics

In [2]:
import duckdb

In [28]:
train_path = 'https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/7/output/user_clicks_train'




In [37]:
df = pd.read_parquet(train_path)


In [38]:
df.shape

(98250, 191)

In [39]:
df.head(2)

,User_UserID,User_UserAgentID,User_UserAgentOSID,User_UserDeviceID,User_UserAgentFamilyID,ph_UserID_count,ph_PhoneRequestDate_min,ph_PhoneRequestDate_max,ph_seconds_since_last,ph_num_events_1d,...,visit_3dv4_change,visit_4dv7_change,visit_7dv14_change,visit_14dv30_change,visit_30dv60_change,visit_60dv90_change,visit_90dv180_change,visit_180dv365_change,visit_365dv730_change,visit_ads_visited_label
0,1727226,7075.0,20.0,2014.0,62.0,2,2015-04-26 11:31:54.0,2015-04-27 16:05:30.0,892470,0.0,...,1.0,0.258929,0.666667,1.0,1.0,1.0,1.0,1.0,1.0,30
1,3407700,18952.0,20.0,2014.0,15.0,<NA>,None,None,<NA>,NaN,...,0.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,10


In [40]:
import duckdb
con = duckdb.connect()

In [41]:
labeldf = con.execute("""
select s.UserID,
max(label.num_click_target) as num_click_target
from '/usr/local/lake/relbench/avito_100k_integ_test/SearchInfo.parquet' s
left join 
(select SearchID,
count(*) AS num_click_target
from '/usr/local/lake/relbench/avito_100k_integ_test/SearchStream.parquet'
where IsClick = 1
and SearchDate > '2015-05-08'
and SearchDate <='2015-05-13'
group by SearchID
) label
on s.SearchID = label.SearchID
group by s.UserID""").df()

In [42]:
df.head(2)

,User_UserID,User_UserAgentID,User_UserAgentOSID,User_UserDeviceID,User_UserAgentFamilyID,ph_UserID_count,ph_PhoneRequestDate_min,ph_PhoneRequestDate_max,ph_seconds_since_last,ph_num_events_1d,...,visit_3dv4_change,visit_4dv7_change,visit_7dv14_change,visit_14dv30_change,visit_30dv60_change,visit_60dv90_change,visit_90dv180_change,visit_180dv365_change,visit_365dv730_change,visit_ads_visited_label
0,1727226,7075.0,20.0,2014.0,62.0,2,2015-04-26 11:31:54.0,2015-04-27 16:05:30.0,892470,0.0,...,1.0,0.258929,0.666667,1.0,1.0,1.0,1.0,1.0,1.0,30
1,3407700,18952.0,20.0,2014.0,15.0,<NA>,None,None,<NA>,NaN,...,0.0,1.000000,1.000000,1.0,1.0,1.0,1.0,1.0,1.0,10


In [43]:
df = df.merge(labeldf, left_on='User_UserID', right_on='UserID', how='left')
df['num_click_target'] = df['num_click_target'].apply(lambda x: 1 if x > 1 else 0)

In [44]:
df['num_click_target'].sum()/len(df)

0.00072264631043257

In [45]:
test = pd.read_parquet(train_path)


In [46]:
test.shape

(98250, 191)